<a href="https://colab.research.google.com/github/mecuryD/story-generator/blob/main/story_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Do it! BERT와 GPT로 배우는 자연어 처리](https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook) 를 많이 참고하였습니다

### **각종 설정하기**

In [ ]:
# 의존성 패키지 설치
!pip install transformers

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
# 모델 환경 설정
import os
import torch
from glob import glob
from dataclasses import dataclass, field

@dataclass
class TrainArguments :
    pretrained_model_name: str = field(default="skt/kogpt2-base-v2")
    downstream_task_name: str = field(default="sentence-generation")
    downstream_model_dir: str = field(default="/content/drive/MyDrive/Do-it-자연어처리/checkpoint-story")
    max_seq_length: int = field(default=32)
    save_top_k: int = field(default=1)
    monitor: str = field(default="min val_loss")
    epochs: int = field(default=3)
    batch_size: int = field(default=96)
    learning_rate: float = field(default=5e-5)
    seed: int = field(default=None, metadata={"help": "random seed."})
    overwrite_cache: bool = field(default=False, metadata={"help": "Overwrite the cached training and evaluation sets"})
    force_download: bool = field(default=False,metadata={"help": "force to download downstream data and pretrained models."})
    test_mode: bool = field(default=False,metadata={"help": "Test Mode enables `fast_dev_run`"})
    cpu_workers: int = field(default=os.cpu_count(),metadata={"help": "number of CPU workers"})
    fp16: bool = field(default=False,metadata={"help": "Enable train on FP16"})

In [ ]:
args = TrainArguments(
    max_seq_length = 32,
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    epochs=3,
    seed=7,
)

### **토크나이저 준비하기**

In [ ]:
# 토크나이저 선언
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>', 
                                                    pad_token='<pad>', mask_token='<mask>')

print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


### **데이터 불러오기 / 전처리**

In [ ]:
import os
import csv
import time
import torch
import logging
from filelock import FileLock
from dataclasses import dataclass
from typing import List, Optional
from torch.utils.data.dataset import Dataset
from transformers import PreTrainedTokenizerFast


@dataclass
class GenerationExample :
  text : str

@dataclass
class GenerationFeatures : 
  input_ids : List[int]
  attention_mask : Optional[List[int]] = None
  token_type_ids : Optional[List[int]] = None
  labels : Optional[List[int]] = None

In [ ]:
def data_collator(features):
    """
    Very simple data collator that:
    - simply collates batches of dict-like objects
    - Performs special handling for potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    - does not do any additional preprocessing
    i.e., Property names of the input object will be used as corresponding inputs to the model.
    See glue and ner for example of how it's useful.
    """

    # In this function we'll make the assumption that all `features` in the batch
    # have the same attributes.
    # So we will look at the first element as a proxy for what attributes exist
    # on the whole batch.
    if not isinstance(features[0], dict):
        features = [vars(f) for f in features]

    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            else:
                batch[k] = torch.tensor([f[k] for f in features], dtype=torch.long)

    return batch

In [ ]:
'''Corpus'''
class StoryCorpus :
  def __init__(self, f_path) :
    # load raw data
    with open(f_path, "r", encoding="utf-8") as f :
      self.data = f.readlines()
    
    # create examples
    self.examples = self._create_examples(self.data)
    self.train = self.examples[:int(len(self.examples)*0.9)]
    self.valid = self.examples[int(len(self.examples)*0.9):]

  def _get_examples(self, mode) :
    if mode == 'train' :
      return self.train
    else : 
      return self.valid

  def _create_examples(self, lines) :
    examples = []
    for (i, line) in enumerate(lines) :
      examples.append(GenerationExample(text=line))
    return examples

'''Convert examples to features'''
def _convert_examples_to_generation_features(
    examples : List[GenerationExample],
    tokenizer, args):
  
  # tokenization
  batch_encoding = tokenizer(
      [example.text for example in examples],
      max_length = args.max_seq_length,
      padding = "max_length",
      truncation = True,
  )

  features = []
  for i in range(len(examples)) :
    inputs = {k:batch_encoding[k][i] for k in batch_encoding}
    feature = GenerationFeatures(**inputs,labels=batch_encoding["input_ids"][i])
    features.append(feature)
  
  for i, example in enumerate(examples[:5]) :
    print("*** Example ***")
    print("sentence : %s" % (example.text))
    print("tokens : %s" % (" ".join(tokenizer.convert_ids_to_tokens(features[i].input_ids))))
    print("features : %s" % features[i])

  return features

'''Dataset'''
class StoryDataset(Dataset) :
  def __init__(self, examples,args, tokenizer,mode,
               convert_examples_to_features_fn = _convert_examples_to_generation_features,):
    if examples is not None : 
      self.examples = examples
    else :
      raise KeyError("examples is not valid")

    # Load data features from dataset file
    print(f"Creating features from {mode}-examples")
    tokenizer.pad_token = tokenizer.eos_token
    self.features = convert_examples_to_features_fn(examples,tokenizer,args)

  def __len__(self) :
    return len(self.features)
  
  def __getitem__(self, i) :
    return self.features[i]

  def get_labels(self):
        return self.corpus.get_labels()

In [ ]:
corpus = StoryCorpus("/content/drive/MyDrive/Do-it-자연어처리/data/grimm/fairy_tale_clear.txt")

# train data
train = corpus._get_examples('train')
print(train[0], "\n\n")
train_features = StoryDataset(train, args, tokenizer,'train', _convert_examples_to_generation_features)

GenerationExample(text='옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.\n') 


Creating features from train-examples
*** Example ***
sentence : 옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.

tokens : ▁옛날 ▁옛 적에 ▁소 망을 ▁이루어 ▁주는 ▁힘이 ▁여전히 ▁통 했던 ▁때에 , ▁딸 들이 ▁모두 ▁예쁜 ▁왕이 ▁한 ▁분 ▁살고 ▁계 셨 어요.
 </s> </s> </s> </s> </s> </s> </s> </s>
features : GenerationFeatures(input_ids=[12346, 10256, 16871, 9077, 12615, 9442, 10764, 15453, 12632, 9107, 9850, 11435, 387, 10265, 9136, 9432, 41775, 11828, 9036, 9129, 12517, 9142, 7816, 14782, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[12346, 10256, 16871, 9077, 12615, 9442, 10764, 15453, 12632, 9107, 9850, 11435, 387, 10265, 9136, 9432, 41775, 11828, 9036, 9129, 12517, 9142, 7816, 14782, 1, 1, 1, 1, 1, 1, 1, 1])
*** Example ***

In [ ]:
# validation data
valid = corpus._get_examples('valid')
print(valid[0], "\n\n")
valid_features = StoryDataset(valid, args, tokenizer,'valid', _convert_examples_to_generation_features)

GenerationExample(text='다음날 아침 한스는 정원으로 나가보았어요.\n') 


Creating features from valid-examples
*** Example ***
sentence : 다음날 아침 한스는 정원으로 나가보았어요.

tokens : ▁다음날 ▁아침 ▁한 스는 ▁정 원으로 ▁나가 보 았 어요.
 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
features : GenerationFeatures(input_ids=[23212, 17339, 9036, 9860, 9040, 10045, 12312, 7652, 7978, 14782, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[23212, 17339, 9036, 9860, 9040, 10045, 12312, 7652, 7978, 14782, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
*** Example ***
sentence : 가시나무들과 수풀들이 온통 제멋대로 자라고 있었어요. 

tokens : ▁가시 나무 들과 ▁수 풀 들이 ▁온통 ▁제멋대로 ▁자라고 ▁있었 어 요. ▁
 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [ ]:
# DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_dataloader = DataLoader(train_features,
                              batch_size = args.batch_size,
                              sampler = RandomSampler(train, replacement=False),
                              collate_fn = data_collator,
                              drop_last = False,
                              num_workers = args.cpu_workers,)
val_dataloader = DataLoader(valid_features, 
                            batch_size = args.batch_size,
                            sampler = SequentialSampler(valid),
                            collate_fn = data_collator,
                            drop_last = True,
                            num_workers = args.cpu_workers,)

### **모델 초기화**

In [ ]:
# 모델 준비
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

### **파인 튜닝**

In [ ]:
!pip install pytorch-lightning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR

# Define Task Module
class StoryGenerator(pl.LightningModule) :
  def __init__(self,model, args) :
    super().__init__()
    self.model = model
    self.args = args

  def configure_optimizers(self) :
    optimizer = optim.Adam(self.parameters(), lr=1e-3)
    scheduler = ExponentialLR(optimizer, gamma=0.9)
    return {'optimizer' : optimizer,
            'scheduler' : scheduler,
    }

  def training_step(self, inputs, batch_idx) :
    outputs = self.model(**inputs)
    self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
    return outputs.loss

  def validation_step(self, inputs, batch_idx) :
    outputs = self.model(**inputs)
    self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
    return outputs.loss

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Define Trainer function
def get_trainer(args,return_trainer_only=True) :
  ckpt_path = os.path.abspath(args.downstream_model_dir)
  os.makedirs(ckpt_path, exist_ok=True)

  # Checkpoint Callback
  checkpoint_callback = ModelCheckpoint(
      dirpath = ckpt_path, 
      filename = '{epoch}-{val_loss:.2f}',
      save_top_k = args.save_top_k,
      mode = args.monitor.split()[0], # "min"
      monitor = args.monitor.split()[1]  # "val_loss"
  )

  # Trainer
  trainer = Trainer(
      max_epochs = args.epochs,
      fast_dev_run = args.test_mode,
      num_sanity_val_steps = None if args.test_mode else 0,
      callbacks = [checkpoint_callback],
      default_root_dir = ckpt_path,

      # For GPU Setup
      deterministic=torch.cuda.is_available() and args.seed is not None,
      gpus = torch.cuda.device_count() if torch.cuda.is_available() else None,
      precision = 16 if args.fp16 else 32,
  )

  if return_trainer_only :
    return trainer
  else : 
    return checkpoint_callback, trainer

In [ ]:
# Model
task = StoryGenerator(model, args)

# Fine-tuning
trainer = get_trainer(args)
trainer.fit(
    task,
    train_dataloaders = train_dataloader,
    val_dataloaders = val_dataloader,
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:382: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
text = '그렇게 그들은 허약한 왕비를 목욕탕으로 데리고 가'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=256,
                           repetition_penalty=3.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

그렇게 그들은 허약한 왕비를 목욕탕으로 데리고 가 말했어요.
</s>
